In [1]:
#pip install DuckDuckGoImages
#import DuckDuckGoImages as ddg

#ddg.download('doberman', folder='./pets/doberman', max_urls=10000)
#ddg.download('labrador', folder='./pets/labrador', max_urls=10000)

import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.applications.resnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [2]:
inputs = keras.Input(shape=(224, 224, 3))

base_model = keras.applications.ResNet50(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
) 

# Freeze the base_model
base_model.trainable = False

# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name, "-", layer.trainable)

model = keras.models.Sequential()
model.add(base_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(2, activation='softmax'))


In [6]:
model.compile(loss='categorical_crossentropy', 
              optimizer=keras.optimizers.RMSprop(), 
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 10)                1003530   
Total params: 24,591,242
Trainable params: 1,003,530
Non-trainable params: 23,587,712
_________________________________________________________________


In [7]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) 
train_data=train_datagen.flow_from_directory('./data/train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) 
test_data=train_datagen.flow_from_directory('./data/test',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

val_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) 
val_data=train_datagen.flow_from_directory('./data/val',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 1188 images belonging to 2 classes.
Found 250 images belonging to 2 classes.
Found 250 images belonging to 2 classes.


In [8]:
step_size_train=train_data.n//train_data.batch_size
step_size_test=test_data.n//test_data.batch_size

model.fit_generator(generator=train_data,
                    steps_per_epoch=step_size_train,
                    epochs=10,
                    validation_data=test_data,
                    validation_steps=step_size_test
                   )

model.save('restnet_pet.h5')

Epoch 1/10


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,10] labels_size=[32,2]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at <ipython-input-8-77a5cfb0ebb8>:4) ]] [Op:__inference_train_function_14602]

Function call stack:
train_function
